In [39]:
import os
from glob import glob
import pandas as pd

path = "/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/data/bidmc-ppg-and-respiration-dataset-1.0.0"
EXT = "*Signals.csv"
all_csv_files = [file for path, subdir, files in os.walk(path) for file in glob(os.path.join(path, EXT))]
patients_bidmc = []
data_bidmc = {}
no_errors = 0
for file in all_csv_files:
    try:
        df = pd.read_csv(file)
        X1, X2, X3, X4 = df[' PLETH'].values, df[' V'].values, df[' AVR'].values, df[' II'].values
        # X = np.concatenate([X1.reshape(len(X1),1),X2.reshape(len(X1),1),X3.reshape(len(X1),1),X4.reshape(len(X1),1)], axis=1)
        
        Y = df[' RESP'].values
        
        patient = int(file.split('/')[-1].split('_')[1])
        patients_bidmc.append(patient)
        data_bidmc[patient] = np.array([X4, Y])
    except:
        no_errors += 1
no_errors

8

In [40]:
import pickle

with open("/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/data/data_full.pickle", "rb") as input_file:
	data_original = pickle.load(input_file)
with open("/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/data/parameters_full.pickle", "rb") as input_file:
	parameters = pickle.load(input_file)
with open("/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/data/patients_full.pickle", "rb") as input_file:
	patients_original = pickle.load(input_file)

In [41]:
def process_data_segment(data, downsampled_window_size, n, patient_indices):
    window_size = int(downsampled_window_size * n)
    overlap = 1 / 2

    windows_ecg = []
    windows_resp = []

    for record_index in patient_indices:
        N = len(data[record_index][0, :])
        max_step = int(N // (window_size * overlap))
        for step in range(1, max_step - 1):
            start_idx = int(step * window_size * overlap)
            end_idx = start_idx + window_size
            recrd_ecg = data[record_index][0, start_idx:end_idx]
            recrd_resp = data[record_index][1, start_idx:end_idx]

            if recrd_ecg.min() < recrd_ecg.max():  # Skip flatlined signal segments
                normalized_ecg = (recrd_ecg - recrd_ecg.min()) / (recrd_ecg.max() - recrd_ecg.min())
                normalized_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                windows_ecg.append(np.float16(normalized_ecg))

                if recrd_resp.min() < recrd_resp.max():
                    normalized_resp = (recrd_resp - recrd_resp.min()) / (recrd_resp.max() - recrd_resp.min())
                    normalized_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_resp.append(np.float16(normalized_resp))

    windows_ecg = np.array(windows_ecg)[:, :, np.newaxis]
    windows_resp = np.array(windows_resp)[:, :, np.newaxis]

    print(windows_ecg.shape)
    print(windows_resp.shape)

    return windows_ecg, windows_resp

In [43]:
from scipy import signal

bidmc_windows_ecg, bidmc_windows_resp = process_data_segment(data_bidmc, 1024, 0.9765625, patients_bidmc)

(5310, 1024, 1)
(5310, 1024, 1)


In [44]:
original_windows_ecg, original_windows_resp = process_data_segment(data_original, 1024, 14.0625, patients_original)

(24981, 1024, 1)
(24976, 1024, 1)


Note 1: original data has much more samples (4 times more)

In [46]:
def print_segment_stats(data, label):
    # Calculate stats for each segment
    means = [segment.mean() for segment in data]
    std_devs = [segment.std() for segment in data]
    mins = [segment.min() for segment in data]
    maxs = [segment.max() for segment in data]

    # Print average stats across segments
    print(f"{label} - Mean of Means: {np.mean(means):.2f}")
    print(f"{label} - Mean of Std Deviations: {np.mean(std_devs):.2f}")
    print(f"{label} - Min of Mins: {np.min(mins)}")
    print(f"{label} - Max of Maxs: {np.max(maxs)}")

print_segment_stats(original_windows_ecg, "Old Dataset ECG")
print_segment_stats(bidmc_windows_ecg, "New Dataset ECG")
print("===")
print_segment_stats(original_windows_resp, "Old Dataset Respiratory")
print_segment_stats(bidmc_windows_resp, "New Dataset Respiratory")


Old Dataset ECG - Mean of Means: 0.30
Old Dataset ECG - Mean of Std Deviations: 0.10
Old Dataset ECG - Min of Mins: -0.11688232421875
Old Dataset ECG - Max of Maxs: 1.0859375
New Dataset ECG - Mean of Means: 0.32
New Dataset ECG - Mean of Std Deviations: 0.15
New Dataset ECG - Min of Mins: -0.10504150390625
New Dataset ECG - Max of Maxs: 1.146484375
===
Old Dataset Respiratory - Mean of Means: 0.57
Old Dataset Respiratory - Mean of Std Deviations: 0.26
Old Dataset Respiratory - Min of Mins: -0.090576171875
Old Dataset Respiratory - Max of Maxs: 1.0908203125
New Dataset Respiratory - Mean of Means: 0.42
New Dataset Respiratory - Mean of Std Deviations: 0.33
New Dataset Respiratory - Min of Mins: -0.01480865478515625
New Dataset Respiratory - Max of Maxs: 1.0146484375
